In [11]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
import scipy.ndimage as ndimage
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F

In [12]:
capture = cv2.VideoCapture('data/video.mov')

n = 0
frames = []
while True:
    successful, next_frame = capture.read()
    if not successful:
        # No more frames to read
        print("Processed %d frames" % n)
        break
    frames.append(next_frame)
    n += 1
# Now we have an image! We can process that as we would.

# We have to give up the file at the end.
capture.release()

Processed 101 frames


In [13]:
height, width, channels = frames[0].shape
height, width, channels


(1920, 1440, 3)

In [14]:
def downsize_frame(frame, factor = 2, sigma = 9):
  frame_blurred = cv2.GaussianBlur(frame, (sigma, sigma), 0)
  return frame_blurred[::factor, ::factor]

factor = 4
downsized_size = (width // factor, height // factor) # w,h
output_path = 'data/output_downsized.mp4'
output_format = cv2.VideoWriter_fourcc('M','P','4','V')
output_fps = 30
downsized_output = cv2.VideoWriter(output_path, output_format, output_fps, downsized_size)
downsized_frames = []

for frame in frames:
  output_frame = downsize_frame(frame, factor=factor)
  downsized_frames.append(output_frame)
  downsized_output.write(output_frame)

downsized_output.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [15]:
# first model
# a bunch of 3x3 convolutions
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

# test = cv2.resize(frames[0], (10, 10))
# test_r = torch.tensor(test[:, :, 0]) # red component
test = torch.from_numpy(np.array(downsized_frames[:10])).float().to(device)

print(test.shape)

print("hi")

Using cpu device
torch.Size([10, 480, 360, 3])
hi


In [16]:
print(test.size())

torch.Size([10, 480, 360, 3])


In [17]:
shaped_test = torch.einsum('ijkl -> lijk', test)
# in_channels, batch size, w, h print(shaped_test.shape)

In [18]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv3d(3, 3, (3, 3, 3), bias=True, padding=1)
        
#         # self.conv2 = nn.Conv2d(3, 3, 3)
#         # add more later?

#     def forward(self, x):
#         identity = x.clone()
#         upsampled = nn.Upsample(scale_factor = 2, mode = 'bicubic')(identity)
#         out = F.relu(self.conv1(x))
#         out += upsampled
#         # out = F.relu(self.conv2(out))
        # rneturn out
net = nn.Sequential(
    nn.Conv3d(3, 16, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(16, 32, (3, 1, 1), padding=(1, 0, 0), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 32, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 32, (3, 1, 1), padding=(1, 0, 0), device=device),
    nn.ReLU(),
    nn.Upsample(scale_factor = 4, mode = 'bicubic'),
    nn.Conv3d(32, 32, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 32, (3, 1, 1), padding=(1, 0, 0), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 16, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(16, 3, (3, 1, 1), padding=(1, 0, 0), device=device)
)
torch.manual_seed(0)
print(shaped_test.size())

# a = test.reshape(test, (1, ...))
# print(test_res)


torch.Size([3, 10, 480, 360])


In [19]:
print(shaped_test.size())



torch.Size([3, 10, 480, 360])


In [20]:
print(shaped_test.size())
print(shaped_test[0])

net_test_result = net(shaped_test)
print(net_test_result)
print(net_test_result.shape)

torch.Size([3, 10, 480, 360])
tensor([[[ 61.,  63.,  63.,  ...,  12.,  13.,  13.],
         [ 63.,  64.,  64.,  ...,  13.,  13.,  12.],
         [ 63.,  64.,  64.,  ...,  13.,  13.,  13.],
         ...,
         [ 35.,  35.,  34.,  ...,  67., 112., 119.],
         [ 35.,  35.,  34.,  ...,  44.,  79., 116.],
         [ 35.,  35.,  33.,  ...,  38.,  50.,  90.]],

        [[ 64.,  60.,  58.,  ...,  12.,  13.,  12.],
         [ 65.,  63.,  59.,  ...,  13.,  13.,  12.],
         [ 65.,  64.,  61.,  ...,  13.,  13.,  13.],
         ...,
         [ 34.,  34.,  34.,  ...,  78., 116., 119.],
         [ 34.,  34.,  34.,  ...,  47.,  90., 119.],
         [ 34.,  34.,  33.,  ...,  40.,  56., 101.]],

        [[ 62.,  63.,  62.,  ...,  12.,  12.,  11.],
         [ 63.,  63.,  63.,  ...,  12.,  12.,  12.],
         [ 64.,  64.,  63.,  ...,  12.,  13.,  12.],
         ...,
         [ 33.,  33.,  33.,  ...,  90., 119., 118.],
         [ 33.,  33.,  33.,  ...,  52., 100., 119.],
         [ 33.,  33.,  

In [21]:
print(net)

Sequential(
  (0): Conv3d(3, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (1): ReLU()
  (2): Conv3d(16, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
  (3): ReLU()
  (4): Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (5): ReLU()
  (6): Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
  (7): ReLU()
  (8): Upsample(scale_factor=4.0, mode='bicubic')
  (9): Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (10): ReLU()
  (11): Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
  (12): ReLU()
  (13): Conv3d(32, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (14): ReLU()
  (15): Conv3d(16, 3, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
)
